In [71]:
import math
import torch
from torch.distributions import normal

%matplotlib inline

# Fitting Gaussian

In [77]:
m = normal.Normal(4.0, 0.5)
sample = m.sample((100,))

In [78]:
sample

tensor([3.9052, 3.8018, 4.7267, 3.2874, 3.8692, 4.8902, 5.2769, 3.9898, 4.0905,
        4.3270, 4.3348, 3.8455, 4.4163, 3.2587, 4.4511, 4.1145, 4.2852, 2.7311,
        3.4649, 3.9961, 4.0940, 4.0473, 3.2010, 4.3445, 3.5395, 3.4755, 4.6836,
        3.9888, 3.4280, 4.4842, 4.3357, 3.0576, 3.9690, 4.3254, 4.6122, 4.1205,
        4.3750, 3.3612, 4.6894, 3.3088, 3.9086, 3.6418, 4.2421, 3.8228, 4.2386,
        3.4394, 3.3265, 4.0155, 4.3897, 4.4121, 4.1101, 3.8423, 4.3691, 4.1557,
        4.1370, 4.2630, 4.0064, 3.1609, 3.8725, 4.0313, 4.4246, 4.5667, 4.2082,
        4.4227, 3.8969, 4.0618, 2.9900, 3.9165, 4.4664, 4.5635, 3.8917, 3.0974,
        3.3490, 4.0303, 4.1494, 4.0518, 4.2698, 3.6119, 4.5849, 4.2388, 4.1246,
        3.7975, 4.6549, 4.7445, 3.7502, 2.9897, 3.9085, 3.7969, 3.1440, 3.8353,
        4.3073, 3.5629, 3.6275, 3.1284, 4.4866, 3.8777, 4.4852, 4.3004, 3.9777,
        4.5596])

In [62]:
log_norm_constant = -0.5 * np.log(2 * np.pi)

def log_gaussian(x, mean=0, logvar=0.):
  """
  Returns the density of x under the supplied gaussian. Defaults to
  standard gaussian N(0, I)
  :param x: (*) torch.Tensor
  :param mean: float or torch.FloatTensor with dimensions (*)
  :param logvar: float or torch.FloatTensor with dimensions (*)
  :return: (*) elementwise log density
  """
  if type(logvar) == 'float':
      logvar = x.new(1).fill_(logvar)

  a = (x - mean) ** 2
  log_p = -0.5 * (logvar + a / logvar.exp())
  log_p = log_p + log_norm_constant

  return log_p

In [63]:
def get_likelihoods(X, mu, logvar, log=True):
  """
  :param X: design matrix (examples, features)
  :param mu: the component means (K, features)
  :param logvar: the component log-variances (K, features)
  :param log: return value in log domain?
      Note: exponentiating can be unstable in high dimensions.
  :return likelihoods: (K, examples)
  """

  # get feature-wise log-likelihoods (K, examples, features)
  log_likelihoods = log_gaussian(
      X[None, :, :], # (1, examples, features)
      mu[:, None, :], # (K, 1, features)
      logvar[:, None, :] # (K, 1, features)
  )

  # sum over the feature dimension
  log_likelihoods = log_likelihoods.sum(-1)

  if not log:
      log_likelihoods.exp_()

  return log_likelihoods

In [ ]:
learning_rate = 0.00002

for t in range(1000):
    NLL = gte_likelihoods(X,)
    NLL.backward()

    if t % 100 == 0:
        print("loglikelihood=", NLL.data.numpy(), "p=", p.data.numpy(), "dL/dp= ", p.grad.data.numpy())

    
    p.data -= learning_rate * p.grad.data
    p.grad.data.zero_()

# Maximum Likelihood for Bernoulli with PyTorch

Let's say that we have 100 samples from a Bernoulli distribution:

In [64]:
import torch
import numpy as np

sample = np.array([ 1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,
        0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,
        1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.])

In [65]:
np.mean(sample)

0.725

Let's now define the probability p of generating 1, and put the sample into a PyTorch Variable:

In [66]:
x = torch.from_numpy(sample).float()
p = torch.rand(1).float().requires_grad_()

We are ready to learn the model using maximum likelihood:

In [67]:
learning_rate = 0.00002
for t in range(1000):
    NLL = -torch.sum(torch.log(x*p + (1-x)*(1-p)) )
    NLL.backward()

    if t % 100 == 0:
        print("loglikelihood=", NLL.data.numpy(), "p=", p.data.numpy(), "dL/dp= ", p.grad.data.numpy())

    
    p.data -= learning_rate * p.grad.data
    p.grad.data.zero_()

loglikelihood= 133.97046 p= [0.527544] dL/dp=  [-158.44556]
loglikelihood= 118.19812 p= [0.6906524] dL/dp=  [-32.15291]
loglikelihood= 117.64539 p= [0.72016996] dL/dp=  [-4.7935333]
loglikelihood= 117.63397 p= [0.72435737] dL/dp=  [-0.64375305]
loglikelihood= 117.63374 p= [0.72491515] dL/dp=  [-0.08511353]
loglikelihood= 117.63378 p= [0.7249888] dL/dp=  [-0.01123047]
loglikelihood= 117.63376 p= [0.7249986] dL/dp=  [-0.00141907]
loglikelihood= 117.63376 p= [0.7249986] dL/dp=  [-0.00141907]
loglikelihood= 117.63376 p= [0.7249986] dL/dp=  [-0.00141907]
loglikelihood= 117.63376 p= [0.7249986] dL/dp=  [-0.00141907]
